<a href="https://colab.research.google.com/github/stravo1/Colab-Notebooks/blob/main/Langchain_OpenRouter_FunctionCall_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-dotenv openai langchain_core

In [ ]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

In [3]:
from typing import Any, List, Mapping, Optional, Union, Sequence, Tuple, Dict, Iterator

from openai import OpenAI
from os import getenv

from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_core.prompt_values import PromptValue
from langchain_core.messages.base import BaseMessage
from langchain_core.runnables.config import RunnableConfig

In [4]:
custom_functions = [
    {
        'name': 'add_to_cart',
        'description': 'Add item to cart',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the item ONLY, not the quantity.'
                },
                'price': {
                    'type': 'integer',
                    'description': 'Price of the item'
                },
                'quantity': {
                    'type': 'integer',
                    'description': 'Quantity or number of the item. Use "1" if no quantity or nmber of item is specified. Convert words to integers if required like "three" to 3. DO NOT SKIP THIS FIELD'
                }
            }
        },
        'required': [
            'name',
            'price',
            'quantity'
        ]
    },
    {
        'name': 'remove_from_cart',
        'description': 'Remove or delete item from cart',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the item'
                }
            }
        },
        'required': [
            'name'
        ]
    },
    {
        'name': 'view_item',
        'description': 'View details of an item in the cart',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': ['string'],
                    'description': 'Name of the item specified.'
                }
            }
        },
        'required': [
            'name'
        ]
    },
     {
        'name': 'view_cart',
        'description': 'View all items in the cart',
        'parameters': {
            'type': 'null'
        }
    },
    {
        'name': 'view_total_price',
        'description': 'View total prices of all items in cart.',
        'parameters': {
            'type': 'null'
        }
    },
]

tools = [
    {
        "type": "function",
        "function": custom_functions[i]
    }
    for i in range(len(custom_functions))
]

In [5]:
cart = {}

def add_to_cart(params):
  name = params["name"]
  cart[name] = {"price": params["price"], "quantity": params["quantity"]}

def remove_from_cart(params):
  cart.pop(params["name"])

def view_cart():
  for key, value in cart.items():
    print(key)
    print(value['price'], value['quantity'])

def view_item(params):
  name = params["name"]
  print(cart[name]["price"], cart[name]["quantity"])

def view_total_price():
  total = 0
  for items in cart.values():
    total+= items['price'] * items['quantity']
  print(total)
  return total

In [6]:
# create an OpenAI API client
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=getenv("API_KEY"),
)

# more models at https://openrouter.ai/models
model = "mistralai/mistral-7b-instruct:nitro"

system_prompt = {
      "role": "system",
      "content": r"You are an AI assistant who has access to various functions and depending on the request you need to call the function with the functoin name and proper arguments in the JSON format {'function_name':'...', 'arguments': {...}} strictly following the types. Arguments are optional in some situations",
}

model_config = {
    "model" : model,
    "tools" : tools,
    "temperature" : 0,
    "tool_choice" : "auto",
    "response_format" : {"type": "json_object"},
    "max_tokens": 1000,
}

# response is delivered at once instead of stremaing
def get_completed_response(prompt):
    completion = client.chat.completions.create(
        **model_config,
        messages = [
            system_prompt,
            {
              "role": "user",
              "content": prompt,
            }
          ],
    )
    return completion.choices[0].message.content

# response is streamed
def get_stream(prompt):
    stream = client.chat.completions.create(
        **model_config,
        stream = True,
        messages = [
            system_prompt,
            {
                "role": "user",
                "content": prompt,
            }
        ]
    )
    return stream;

# inherit the LLM class from langchain
class OpenRouter_FunctionCalling(LLM):

    @property
    def _llm_type(self) -> str:
        return "open-router"

    # implementing the _call function is mandatory
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        return "gg"

    def invoke(
        self,
        input: str,
        config: Optional[RunnableConfig] = None, *,
        stop: Optional[List[str]] = None,
        **kwargs: Any
    ) -> str:
        return get_completed_response(input)

    def stream(
        self,
        input: str,
        config: Optional[RunnableConfig] = None, *,
        stop: Optional[List[str]] = None,
        **kwargs: Any
        ) -> Iterator[str]:
        return get_stream(input)


In [9]:
# create an OpenAI API client
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=getenv("API_KEY"),
)

# more models at https://openrouter.ai/models
model = "google/gemma-7b-it:nitro"

system_prompt_assistant = {
      "role": "system",
      "content": r"You are a helpful AI assistant and answer questions precisely and within a few sentences.",
}

model_config_assistant = {
    "model" : model,
    "temperature" : 0.2,
    "max_tokens": 2000,
}

# response is delivered at once instead of stremaing
def get_completed_response_assistant(prompt):
    completion = client.chat.completions.create(
        **model_config_assistant,
        messages = [
            system_prompt_assistant,
            {
              "role": "user",
              "content": prompt,
            }
          ],
    )
    return completion.choices[0].message.content

# response is streamed
def get_stream_assistant(prompt):
    stream = client.chat.completions.create(
        **model_config_assistant,
        stream = True,
        messages = [
            system_prompt_assistant,
            {
                "role": "user",
                "content": prompt,
            }
        ]
    )
    return stream;

# inherit the LLM class from langchain
class OpenRouter_Assistant(LLM):

    @property
    def _llm_type(self) -> str:
        return "open-router"

    # implementing the _call function is mandatory
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        return "gg"

    def invoke(
        self,
        input: str,
        config: Optional[RunnableConfig] = None, *,
        stop: Optional[List[str]] = None,
        **kwargs: Any
    ) -> str:
        return get_completed_response_assistant(input)

    def stream(
        self,
        input: str,
        config: Optional[RunnableConfig] = None, *,
        stop: Optional[List[str]] = None,
        **kwargs: Any
        ) -> Iterator[str]:
        return get_stream_assistant(input)


In [11]:
import json

llm_function_call = OpenRouter_FunctionCalling()
llm_assistant = OpenRouter_Assistant()

instruction = input()
while(instruction != "STOP"):
  if instruction == "summary":
    cart_details = str(cart)
    for chunk in llm_assistant.stream("Tell me about my personality and my interests by looking at my cart details: " + cart_details):
      print(chunk.choices[0].delta.content or "", end="")
    instruction = input()
    continue
  resp = json.loads(llm_function_call.invoke(instruction))
  print(resp)
  func_name = resp["function_name"]
  if "arguments" in resp:
    args = resp["arguments"]
  if func_name == "add_to_cart":
    add_to_cart(args)
  elif func_name == "remove_from_cart":
    remove_from_cart(args)
  elif func_name == "view_cart":
    view_cart()
  elif func_name == "view_item":
    view_item(args)
  elif func_name == "view_total_price":
    view_total_price()
  else:
    print("Invalid function call: ", resp)
  instruction = input()


add 5 lipsticks for $5 each
{'function_name': 'add_to_cart', 'arguments': {'name': 'lipstick', 'price': 5, 'quantity': 5}}
add 3 pairs of eye lashes, $2 dollars each
{'function_name': 'add_to_cart', 'arguments': {'name': 'eye lashes', 'price': 2, 'quantity': 3}}
add laptop for $200
{'function_name': 'add_to_cart', 'arguments': {'name': 'laptop', 'price': 200, 'quantity': 1}}
add two phones $300 each
{'function_name': 'add_to_cart', 'arguments': {'name': 'phones', 'price': 300, 'quantity': 2}}
add 2 bottles for $10
{'function_name': 'add_to_cart', 'arguments': {'name': 'bottles', 'price': 10, 'quantity': 2}}
remove bottles
{'function_name': 'remove_from_cart', 'arguments': {'name': 'bottles'}}
info about lipstick
{'function_name': 'view_item', 'arguments': {'name': 'lipstick'}}
5 5
view cart
{'function_name': 'view_cart'}
laptop
200 1
mobile phones
300 2
lipstick
5 5
eye lashes
2 3
phones
300 2
give me the bill
{'function_name': 'view_total_price'}
1431
summary
## Based on your cart det